### Real Time ASL Fingerspelling
by Loic Baum

In [ ]:
import keras.models
from keras.models import load_model
import cv2
import numpy as np

In [ ]:
#Loading model saved on disk
def load_model_from_disk(MODEL_PATH, MODEL_WEIGHTS_PATH):
    '''A convenience method for re-running certain parts of the
    analysis locally without refitting all the data.'''   
    print('Retrieving model from disk...')
    model = load_model(MODEL_PATH)           
    print('Loading CNN model weights from disk...')
    model.load_weights(MODEL_WEIGHTS_PATH)
    return model
  
model_file = "./saved_nets_weights/cnn-model-vgg.h5"  
weights_file = "./saved_nets_weights/cnn-model-vgg.weights.h5"

model =  load_model_from_disk(model_file,weights_file)
model.summary()

### ASL Alphabet
<img src="https://www.nidcd.nih.gov/sites/default/files/Content%20Images/NIDCD-ASL-hands-2014.jpg" alt='"ASL" spelled out in American Sign Language fingerspelling' style="width: 300px;"/>

In [4]:
map_characters = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J', 10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'O', 15: 'P', 16: 'Q', 17: 'R', 18: 'S', 19: 'T', 20: 'U', 21: 'V', 22: 'W', 23: 'X', 24: 'Y', 25: 'Z', 26: 'del', 27: 'nothing', 28: 'space', 29: 'other'}
# map_characters = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I',  9: 'K', 10: 'L', 11: 'M', 12: 'N', 13: 'O', 14: 'P', 15: 'Q', 16: 'R', 17: 'S', 18: 'T', 19: 'U', 20: 'V', 21: 'W', 22: 'X', 23: 'Y'}

cap = cv2.VideoCapture(0)

last_pred = 'A'
smoothed_pred = 'A'
pred_cnt = 0

while True:
    ret, frame = cap.read()
    frame = frame[0:480, 100:580] #Cropping (480x720 to 480x480)
    frame_resized = cv2.resize(frame,(64,64)) #resizing and keeping aspect_ratio
    frame_resized = frame_resized.astype('float32')/255.0

    prediction = model.predict(frame_resized.reshape(1,64,64,3))
    y_pred = map_characters[int(np.argmax(prediction, axis=1))]
    #Smoothing prediction
    if y_pred == last_pred:
        pred_cnt +=1
    else:
        last_pred = y_pred
        pred_cnt = 0    
    if  pred_cnt >= 5:
        smoothed_pred = y_pred
        
    cv2.putText(frame, smoothed_pred, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 0, 0), 5)
    cv2.imshow("frame",frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()